In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

agg_match_db = create_engine('sqlite:///agg_match.db')
df = pd.read_sql_query('select match_id, player_kills, player_dbno, player_assists, total_dist, team_placement from amatch_solo where total_dist < 8400 and team_placement < 70 order by match_id', agg_match_db)
df['skill'] = df['player_kills']+df['player_dbno']+0.5*df['player_assists']

In [9]:
df1 = df[['match_id', 'skill', 'total_dist', 'team_placement']]

In [ ]:
ids = df1['match_id']
pre_id = ids[0]
dist_sum = 0
skill_sum = 0
res = []
match_len = len(ids)
dist = df1['total_dist']
skill = df1['skill']
for i in range(match_len):
    current_id = ids[i]
    if(current_id == pre_id):
        dist_sum += dist[i]
        skill_sum += skill[i]
    else:
        res.append((pre_id, dist_sum, skill_sum))
        pre_id = current_id
        dist_sum = dist[i]
        skill_sum = skill[i]

In [ ]:
current_id = res[0][0]
state = 0
for i in range(len(df1['match_id'])):
    if df1.loc[i, 'match_id'] == current_match_id:
        df1.loc[i, 'total_dist'] = df1.loc[i, 'total_dist']/total_dist_sum[state][1]
        df1.loc[i, 'final_score'] = df1.loc[i, 'final_score']/total_score_sum[state][1]
    else:
        state += 1
        current_match_id = total_dist_sum[state][0]
        df1.loc[i, 'total_dist'] = df1.loc[i, 'total_dist']/total_dist_sum[state][1]
        df1.loc[i, 'final_score'] = df1.loc[i, 'final_score']/total_score_sum[state][1]